# DVA263 project

# Libraries

Declaring all libraries in use

In [1]:
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2
import os
import cnn as cnn
import preprocessing as pre
import visualization as vis
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

from Project.Code.preprocessing import convert_to_numpy



# Load Dataset

In [2]:
def load_cnn():
    """Load the datasets for the CNN input"""
    CNN_train = "../dataset/CNN_Input/train/"
    CNN_test = "../dataset/CNN_Input/test/"
    CNN_train = pre.import_images(CNN_train, (150,150),batch_size=16, dataset=True)
    CNN_test = pre.import_images(CNN_test, (150,150), batch_size=16,  dataset=True)
    return CNN_train, CNN_test

In [3]:
def load(image_size):
    train_path = "../dataset/Project/train/"
    test_path = "../dataset/Project/test/"

    CNN_train = pre.import_images(train_path, image_size, dataset=True)
    ALL_test = pre.import_images(test_path, image_size, dataset=True)
    
    print(ALL_test)
    y_test, X_test = convert_to_numpy(ALL_test)
    return y_test, X_test

In [4]:
image_size = (152, 152)

y_test, X_test = load(image_size)

print(y_test.shape)
print(X_test.shape)

#for label in y_test: 
    #print(label)

Found 259 files belonging to 4 classes.
Found 33 files belonging to 4 classes.
<_PrefetchDataset element_spec=(TensorSpec(shape=(152, 152, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4,), dtype=tf.float32, name=None))>
(132,)
(5016, 152, 3)


2025-01-20 16:11:43.924635: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# Prepare Data

In [5]:
def prep_cnn(cnn_train, cnn_test):
    """Add noise images to train and test set and normalize"""
    g_noisy_train_data = cnn_train.map(pre.add_gaussian_noise)
    snp_noisy_train_data = cnn_train.map(pre.add_salt_and_pepper_noise)
    combined_train_data = cnn_train.concatenate(g_noisy_train_data).concatenate(snp_noisy_train_data)

    percentage_noisy_images = 0.2
    total_test_images = len(CNN_test) * 16  # Total images in test data
    num_noisy_images = int(total_test_images * percentage_noisy_images)

    g_noisy_test_data = cnn_test.take(num_noisy_images).map(pre.add_gaussian_noise)
    snp_noisy_test_data = cnn_test.take(num_noisy_images).map(pre.add_salt_and_pepper_noise)
    remaining_test_data = cnn_test.skip(num_noisy_images)

    combined_test_data = g_noisy_test_data.concatenate(remaining_test_data)
    combined_test_data = combined_test_data.concatenate(snp_noisy_test_data)

    test_data_norm, train_data_norm = pre.normalize_dataset(combined_train_data, combined_test_data)
    return train_data_norm, test_data_norm


# Build and Compile Model

In [6]:
def build_cnn():
    model = cnn.build_cnn_model()
    return model

#  Train Model

In [7]:
def train_cnn(model,cnn_train, cnn_test):
    history = cnn.train_model(model, cnn_train, cnn_test)
    return history

# Putting everything together

Main section putting each part together and running the classification task

In [8]:
CNN_train, CNN_test = load_cnn()
CNN_train, CNN_test = prep_cnn(CNN_train, CNN_test)

CNN_model = build_cnn()
CNN_history = train_cnn(CNN_model, CNN_train, CNN_test)
CNN_model.predict(CNN_test)


Found 83 files belonging to 4 classes.
Found 62 files belonging to 4 classes.
Epoch 1/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 133ms/step - accuracy: 0.2303 - loss: 1.4082 - val_accuracy: 0.3145 - val_loss: 1.3836 - learning_rate: 0.0010
Epoch 2/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 135ms/step - accuracy: 0.3379 - loss: 1.3861 - val_accuracy: 0.2581 - val_loss: 1.3812 - learning_rate: 0.0010
Epoch 3/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 134ms/step - accuracy: 0.2431 - loss: 1.3850 - val_accuracy: 0.2581 - val_loss: 1.3798 - learning_rate: 0.0010
Epoch 4/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 152ms/step - accuracy: 0.2689 - loss: 1.3828 - val_accuracy: 0.4032 - val_loss: 1.3781 - learning_rate: 0.0010
Epoch 5/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 183ms/step - accuracy: 0.3455 - loss: 1.3772 - val_accuracy: 0.3468 - val_loss: 1.3715 - learning_rate: 0.0010
Epoch 6/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 153ms/step - accuracy: 0.4974 - loss: 1.3701 - val_accuracy: 0.3952 - val_loss: 1.3557 - learning_rate: 0.0010
Epoch 7/70

array([[7.36002925e-10, 9.42955580e-07, 9.99944687e-01, 5.43649039e-05],
       [1.65185134e-03, 9.97207642e-01, 1.00395096e-06, 1.13952963e-03],
       [9.99999881e-01, 7.27510781e-08, 1.33138865e-11, 1.91345231e-10],
       [9.99824405e-01, 1.66868529e-04, 8.65627771e-06, 6.11668014e-08],
       [3.74177966e-06, 2.29517682e-04, 9.50575829e-01, 4.91909608e-02],
       [1.57624947e-06, 2.63335587e-05, 5.88083733e-03, 9.94091213e-01],
       [9.33775300e-05, 2.58478976e-04, 2.54489738e-03, 9.97103333e-01],
       [3.16587370e-03, 9.96834040e-01, 4.01698314e-10, 2.36292923e-08],
       [1.00000000e+00, 3.38314265e-08, 1.50796996e-12, 3.11214698e-08],
       [1.00499536e-04, 9.97347593e-01, 8.48734146e-07, 2.55109323e-03],
       [6.01498641e-06, 1.43703789e-06, 9.99867439e-01, 1.24974482e-04],
       [2.88604915e-05, 2.77327839e-04, 2.31757178e-03, 9.97376323e-01],
       [1.07603114e-06, 7.83293799e-05, 6.13517361e-04, 9.99307096e-01],
       [1.03580739e-07, 4.25575172e-06, 2.17152364e